In [25]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

X, y = load_diabetes(return_X_y = True)

columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [4]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [5]:
# print the coefficients
print (linreg.intercept_)
print (linreg.coef_)

152.5381335195406
[ -35.55683674 -243.1692265   562.75404632  305.47203008 -662.78772128
  324.27527477   24.78193291  170.33056502  731.67810787   43.02846824]


In [9]:
# calculate the R-squared value for the model
y_pred_train = linreg.predict(X_train)
metrics.r2_score(y_train, y_pred_train)

0.5539285357415583

In [10]:
# calculate the R-squared value for the model
y_pred_test = linreg.predict(X_test)
metrics.r2_score(y_test, y_pred_test)

0.33222203269065176

In [13]:
from sklearn.externals import joblib

joblib.dump(value=linreg, filename='diabetes_linreg_model.pkl')

['diabetes_linreg_model.pkl']

In [17]:
from azureml.core.model import Model
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
model = Model.register(model_path="diabetes_linreg_model.pkl",
                       model_name="diabetes_linreg_model.pkl",
                       tags={'area': "diabetes", 'type': "regression"},
                       description="Ridge linreg model to predict diabetes",
                       workspace=ws)

Registering model diabetes_linreg_model.pkl


In [27]:
%%writefile score.py
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression
from azureml.core.model import Model
import json
import numpy

def init():
    global model
    model_path = Model.get_model_path('diabetes_linreg_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any datatype if it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


In [28]:
from azureml.core.conda_dependencies import CondaDependencies 
myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [29]:
from azureml.core.image import Image, ContainerImage
image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "diabetes", 'type': "regression"},
                                 description = "Image with linear regression model")
image = Image.create(name = "diabeteslinregimage1",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

Creating image
Running..................................
Succeeded
Image creation operation finished for image diabeteslinregimage1:2, operation "Succeeded"


In [30]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                          memory_gb = 1, 
                                          tags = {"data": "diabetes", "type": "regression"}, 
                                          description = 'Diabetes with linear regression model')

In [57]:
from azureml.core.webservice import Webservice
service_name = 'diabetes-linreg3'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)
service.wait_for_deployment(show_output = True)

Creating service
Running.............
SucceededACI service creation operation finished, operation "Succeeded"


In [58]:
import json
test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})
test_sample = bytes(test_sample,encoding = 'utf8')
prediction = service.run(input_data = test_sample)
print(prediction)

[9724.011849113987, 4009.9374656088485]


In [55]:
print(service.scoring_uri)

None


In [49]:
response.content

b'{"message": "Expects Content-Type to be application/json"}'

In [ ]:
services = Webservice.list(ws)
for k in services:
    print(k.scoring_uri)

In [60]:
#To get service by name
servicelinreg3 = Webservice(workspace=ws, name = "diabetes-linreg3")
prediction = service.run(input_data = test_sample)
print(prediction)

[9724.011849113987, 4009.9374656088485]


In [ ]:
servicelinreg3.scoring_uri

In [66]:
import requests
import json

# URL for the web service
scoring_uri = servicelinreg3.scoring_uri
# If the service is authenticated, set the key
#key = '<your key>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [1,2,3,4,5,6,7,8,9,10], 
            [10,9,8,7,6,5,4,3,2,1]
        ]
       }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

[9724.011849113987, 4009.9374656088485]
